## Description:
We'll begin the project by first building a primary classification model to classify the images into one of the 3 main categories only. (i.e. Glasses and Sunglasses, Trousers and Jeans, and Shoes).

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

from os import environ # To get the file path of the datasets location from Windows environment variables.

In [2]:
# load the datasets from the path stored in the operating system's environment variable
dataset_path = environ.get('CNN_Practical_Dataset_Path')

data_train = np.load(dataset_path + "/Primary categories - Train.npz")
data_val = np.load(dataset_path + "/Primary categories - Validation.npz")
data_test = np.load(dataset_path + "/Primary categories - Test.npz")

In [3]:
list(data_train.keys())

['images', 'labels']

In [4]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [5]:
# Scaling the pixel values of all images
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [6]:
images_train.shape, images_val.shape, images_test.shape,

((12963, 120, 90, 3), (1620, 120, 90, 3), (1620, 120, 90, 3))

In [7]:
# Defining constants
EPOCHS = 15
BATCH_SIZE = 64

In [8]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([32,64]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs_Practical/Model Primary_Classification/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [17]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of the CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #from_logits=True incorporates Softmax into the loss function.

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    # Defining the logging directory
    log_dir = "Logs_Practical\\Model Primary_Classification\\fit\\" + "run-{}".format(session_num)
    
    # Defining function to plot the confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
       
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses', 'Trousers/Jeans', 'Shoes'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models\Model Primary_Classification\Run-{}".format(session_num))
    
    return accuracy

In [18]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
raise StopIteration("Stop automatic re-run of all cells here.")

"""This is here to stop the notebook from running the cells after this point when automatically re-running all cells.
This is to prevent re-training the model. It has already been trained and saved to an external file.
The model will be loaded instead.""";

In [12]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_FILTER_NUM: filter_num
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs_Practical/Model Primary_Classification/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1

--- Starting trial: run-1
{'filter_size': 3, 'filters_number': 32}
Epoch 1/15
203/203 - 68s - loss: 0.0748 - accuracy: 0.9797 - val_loss: 0.0105 - val_accuracy: 0.9969
Epoch 2/15
203/203 - 65s - loss: 0.0108 - accuracy: 0.9987 - val_loss: 0.0037 - val_accuracy: 0.9994
Epoch 3/15
203/203 - 65s - loss: 0.0076 - accuracy: 0.9992 - val_loss: 0.0045 - val_accuracy: 0.9981
Epoch 4/15
203/203 - 65s - loss: 0.0051 - accuracy: 0.9992 - val_loss: 0.0029 - val_accuracy: 0.9988
Epoch 5/15
203/203 - 65s - loss: 0.0095 - accuracy: 0.9981 - val_loss: 0.0033 - val_accuracy: 0.9981
Epoch 6/15
203/203 - 65s - loss: 0.0093 - accuracy: 0.9985 - val_loss: 0.0031 - val_accuracy: 0.9988
51/51 [==============================] - 2s 49ms/step - loss: 0.0029 - accuracy: 0.9988
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:

KeyboardInterrupt: 

In [ ]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models\Model Primary_Classification\Run-1")

In [ ]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

In [ ]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

In [3]:
%load_ext tensorboard
%tensorboard --logdir "Logs_Practical/Model Primary_Classification/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5668), started 0:01:47 ago. (Use '!kill 5668' to kill it.)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs_Practical/Model Primary_Classification/fit"